<a href="https://colab.research.google.com/github/Shama09/files/blob/master/ML2_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

In [ ]:
import lightgbm as lgbm

In [ ]:
with open('sample_data/additional_features.json') as f:
    ad_data = json.load(f)

In [ ]:
categories = []
df_index = []

for i in range(len(ad_data)):
  categories.append(ad_data[i]['categories'])
  df_index.append(ad_data[i]['index'])

In [ ]:
df_cat = pd.DataFrame(columns=['categories'])
for i in range(df.shape[0]):
  df_cat.at[i, 'categories'] = []

In [ ]:
for i in range(len(df_index)):
  df_cat.at[df_index[i], 'categories'] = categories[i]

In [ ]:
df_cat.to_csv('cat.csv', index=False)

In [ ]:
#df = pd.read_json('time_series_train.json', orient='columns')

# Data to csv

In [ ]:
#a, b = df.shape
#data = np.empty((a, b))
#data[:] = np.nan

for i in range(a):
    for j in range(b):
        if (df.iloc[i][j] == None):
            break
        data[i, df.iloc[i][j][0]] = df.iloc[i][j][1]

In [ ]:
#pd.DataFrame(data).to_csv('dataml2hw1.csv',  header=None, index=None)

# Analysis

In [5]:
df = pd.read_csv('dataml2hw1.csv', header=None)

In [7]:
n = 10

In [90]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(df.iloc[:, -n:].T).T

In [91]:
X[0]

array([60247., 60247., 60247., 60247., 60247., 60247., 60247., 60247.,
       24850., 95644.])

In [6]:
#df.shape[0] - df.iloc[:, 90:].isna().any(axis=1).sum()

25439

In [ ]:
#train = df[~df.iloc[:, 90:].isna().any(axis=1)].iloc[:, 90:].values

In [ ]:
#X_train, X_test = train_test_split(train, test_size=0.2, random_state=42)

In [92]:
x = X[:, :n-1]
y = X[:, n-1]
#for i in range(1, X_train.shape[1] - n):
#    x = np.concatenate((x, X_train[:, i : n+i]), axis=0)
#    y = np.concatenate((y, X_train[:, i + n]))


In [75]:
#Model - delta

In [93]:
train_delta = x[:, 1:] - x[:, :-1]

In [94]:
y_delta = y - x[:, -1]

In [95]:
model_c = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=10, random_state=0)

In [106]:
model_r = GradientBoostingRegressor(n_estimators=10, learning_rate=1.0, max_depth=30, random_state=0)

In [96]:
model_c.fit(train_delta[10000:15000], y_delta[10000:15000])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [107]:
model_r.fit(train_delta[10000:15000], y_delta[10000:15000])

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=30,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=10,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [98]:
print(model_c.score(train_delta, y_delta))
print(mean_absolute_error(model_c.predict(train_delta), y_delta))

0.4272270596115204
94.08687206965841


In [109]:
print(model_r.score(train_delta, y_delta))
print(mean_absolute_error(model_r.predict(train_delta), y_delta))

0.028768498668371615
99.17530942672835


In [26]:
#model delta

In [99]:
pred1 = model_c.predict(X[:, 2:] - X[:, 1:-1])
pred2 = model_c.predict(np.concatenate((X[:, 3:] - X[:, 2:-1], pred1.reshape(-1, 1)), axis=1))
pred3 = model_c.predict(np.concatenate((X[:, 4:] - X[:, 3:-1], pred1.reshape(-1, 1), pred2.reshape(-1, 1)), axis=1))

In [68]:
pred1 = model_r.predict(X[:, 2:] - X[:, 1:-1])
pred2 = model_r.predict(np.concatenate((X[:, 3:] - X[:, 2:-1], pred1.reshape(-1, 1)), axis=1))
pred3 = model_r.predict(np.concatenate((X[:, 4:] - X[:, 3:-1], pred1.reshape(-1, 1), pred2.reshape(-1, 1)), axis=1))

In [101]:
pred = np.vstack((X[:, -1] + pred1, X[:, -1] + pred1 + pred2, X[:, -1] + pred1 + pred2 + pred3))
pred = np.ravel(pred.T)
pred

array([9.56450e+04, 9.56440e+04, 1.02118e+05, ..., 1.00000e+00,
       0.00000e+00, 0.00000e+00])

In [76]:
# model - mean

In [121]:
train_mean = X - X.mean(axis=1, keepdims=True)

In [147]:
train_mean.shape

(29860, 10)

In [140]:
x_mean = train_mean[:, :-1]
y_mean = train_mean[:, -1]

In [143]:
model_cm = GradientBoostingRegressor(n_estimators=10, learning_rate=1.0, max_depth=10, random_state=0)

In [144]:
model_cm.fit(x_mean[10000:15000], y_mean[10000:15000])

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=10,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=10,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [145]:
print(model_cm.score(x_mean, y_mean))
print(mean_absolute_error(model_cm.predict(x_mean), y_mean))

-0.3463492868958511
102.6077518548944


In [148]:
pred1 = model_cm.predict(train_mean[:, 1:])
pred2 = model_cm.predict(np.concatenate((train_mean[:, 2:], pred1.reshape(-1, 1)), axis=1))
pred3 = model_cm.predict(np.concatenate((train_mean[:, 3:], pred1.reshape(-1, 1), pred2.reshape(-1, 1)), axis=1))

In [152]:
pred = np.vstack((X.mean(axis=1) + pred1, X.mean(axis=1) + pred1 + pred2, X.mean(axis=1) + pred1 + pred2 + pred3))
pred = np.ravel(pred.T)
pred

array([5.68358018e+04, 5.80105743e+04, 5.53322654e+04, ...,
       9.97170077e-01, 1.20330653e+00, 1.05879757e+00])

To file

In [153]:
Id = []
for i in range(df.shape[0]):
    Id.append(str(i)+str('_')+str(0))
    Id.append(str(i)+str('_')+str(1))
    Id.append(str(i)+str('_')+str(2))

In [35]:
index = pd.DataFrame(Id, columns=['Id'])

In [154]:
index['Predicted'] = np.abs(pred.astype(int))

In [155]:
index

,Id,Predicted
0,0_0,56835
1,0_1,58010
2,0_2,55332
3,1_0,1
4,1_1,2
...,...,...
89575,29858_1,3
89576,29858_2,6
89577,29859_0,0
89578,29859_1,1


In [157]:
index.to_csv('pred.csv', index=False)